## CNN 

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import Functions as func
import var
import Viz
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using plaidml.keras.backend backend.


In [2]:
### best is model weights 2

In [3]:
def get_conv_model_normal(activation = 'relu', dropout = .25, kernal_reg = regularizers.l1(), kern_init = 'he_uniform', 
                         pad = 'same', opt = 'adam'):
    
    inp_shape = (96,96,3)
   
#     kernal_reg = regularizers.l1(kern_lr)
    dil_rate = 2
#     optimizer = Adam(lr = opt_lr)
    
    model = Sequential() 
    
    model.add(Conv2D(64, kernel_size=(3,3),activation=activation, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = kern_init,  padding = pad, name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    
    
    model.add(Conv2D(64, (3, 3), activation=activation, kernel_regularizer = kernal_reg, 
                     kernel_initializer = kern_init,padding = pad))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))
    

    
    model.add(Conv2D(128, (3, 3), activation=activation, kernel_regularizer = kernal_reg, 
                     kernel_initializer = kern_init, padding = pad))
    model.add(Conv2D(128, (3, 3), activation=activation, kernel_regularizer = kernal_reg, 
                     kernel_initializer = kern_init, padding = pad))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))

    
    model.add(Flatten())

    
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))

    model.add(Dropout(dropout))

    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    return model 


In [4]:
x_train, x_test, y_train, y_test = func.get_samples('normal')
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(6512, 96, 96, 3) (724, 96, 96, 3)
(6512, 3) (724, 3)


In [ ]:
# param_grid = {
#     'activation': 'relu tanh sigmoid hard_sigmoid linear'.split(), 
#     'momentum': [0.0, .2, .4, .6, .8, .9], 
#     'learn_rate': [.001, .01, .1, .2, .3], 
#     'dropout_rate': [0.0, .1, .2, .3, .4, .5, .6, .7, .8, .9], 
#     'weight_contraint': [1, 2, 3, 4, 5], 
#     'neurons': [1,5,10,15,20,25,30],
#     'init': 'uniform lecun_uniform normal zero glorot_normal glorot_uniform he_normal he_uniform'.split(), 
#     'optimizer': 'SGD RMSprop Adagrad Adadelta Adam Adamax Nadam'.split(), 
#     'epochs': [25], 
#     'batch_size': [16, 32, 64],     
# }

param_grid = {
    'activation': 'relu'.split(), 
    'dropout': [0.0, .3, .6, .9], 
    'epochs': [25], 
    'batch_size': [16, 32, 64],  
    'opt': ['Adam', 'Nadam'], 
    'kernal_reg': [regularizers.l1(.001), regularizers.l2(.001)], 
    'pad': ['same'], 
    'kern_init': 'he_uniform'.split()

}


normal_model = KerasClassifier(build_fn = get_conv_model_normal, batch_size = 16, epochs = 20)

grid = GridSearchCV(estimator = normal_model, param_grid = param_grid, verbose = 2, n_jobs = 12, cv = 2)

grid.fit(x_train, y_train)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


In [ ]:
pickle.dump(grid, open('../Pickles/GridSearchCNN.p', 'wb'))

In [ ]:
assert False

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


augment = True 
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=14)
model_checkpoint = ModelCheckpoint('../models/CNN-ModelCheckpointWeights3.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 3, mode = 'min')
epochs = 100
batch_size = 16
if var.img_type == 'grey': 
    dim = (x_train.shape[1], x_train.shape[2], 1)
else: 
    dim = (x_train.shape[1], x_train.shape[2], 3)
    
normal_model = get_conv_model_normal(dim =dim)

if augment: 
    augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                           horizontal_flip = True, shear_range = .15, 
                                     fill_mode = 'nearest', zoom_range = .15)
    augmentation.fit(x_train)
    normal_history = normal_model.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)
else: 
    
    normal_history = normal_model.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


In [ ]:
print(normal_model.summary())

### Loss & Accuracy

In [ ]:
Viz.plot_loss_accuracy(normal_history)

### AUC and ROC Curves

In [ ]:
if var.img_type == 'grey': 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights3.h5') #load the best weights before overfitting
 


    
Viz.plot_roc_auc(normal_model, x_test, y_test)

### Confusion Matrix

In [ ]:
if var.img_type == 'grey': 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
        
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights3.h5') #load the best weights before overfitting





y_test_prob = [np.argmax(i) for i in normal_model.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in normal_model.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'])


In [ ]:
assert False

#### Test on Google Data


In [ ]:
import cv2
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
base_path = '../Tests/Photos'
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights3.h5')

for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    img = func.get_image_value(full_path, var.norm_dimension, var.img_type)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    pred = normal_model.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')

In [ ]:
#model 3
if var.img_type == 'grey': 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
        

img = func.get_image_value('../Tests/Photos/AR.jpg', var.norm_dimension, var.img_type)

explainer = lime_image.LimeImageExplainer()

explanation = explainer.explain_instance(img, normal_model.predict, top_labels = 5, hide_color = 0, 
                                         num_samples = 1000)

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only = False,
                                           num_features = 10, hide_rest = False)
plt.imshow(mark_boundaries(temp/2 + .5, mask))

In [ ]:
Functions.get_img_prediction_bounding_box('../Tests/Photos/Pistol3.jpg', normal_model, var.norm_dimension)

In [ ]:
assert False

In [ ]:
if var.img_type == 'grey': 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
        
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights2.h5')

img = cv2.imread('../TestImages/AR.jpg')
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
rects = ss.process() 

windows = []
locations = []
for x, y, w,h in rects: 
    startx = x 
    starty = y 
    endx = x+w 
    endy = y+h 
    roi = img[starty:endy, startx:endx]
    roi = cv2.resize(roi, dsize =var.norm_dimension, interpolation = cv2.INTER_CUBIC)
    windows.append(roi)
    locations.append((startx, starty, endx, endy))
    


In [ ]:
windows = np.array(windows[:2000])

predictions = normal_model.predict(windows)

In [ ]:
# for i in predictions:
#     print(i)

In [ ]:
clone = img.copy()
new_loc = [] 
new_prob = [] 
no_gun = predictions[:, 0].sum()
handgun = predictions[:,1].sum()
rifle = predictions[:,2].sum()
sum_array = np.array([no_gun, handgun, rifle])
index_unit = np.argmax(sum_array)
cat_types = ['No Gun', 'Handgun', 'Rifle']
print(index_unit)
print(no_gun, handgun, rifle)
new_locations = []
for idx, i in enumerate(predictions):
    category = i[index_unit]
    if category > .5:
        startx, starty, endx, endy = locations[idx]
        new_locations.append([startx, starty, endx, endy])
        new_loc.append(locations[idx])
        new_prob.append(category)

new_locations = np.array(new_locations)

startx = int(new_locations[:,0].mean())
starty =int( new_locations[:,1].mean())
endx = int(new_locations[:,2].mean())
endy = int(new_locations[:,3].mean())


# startx = int(np.median (new_locations[:,0]))
# starty =int( np.median (new_locations[:,1]))
# endx = int(np.median (new_locations[:,2]))
# endy = int(np.median(new_locations[:,3]))

prob_avg = np.array(new_prob).mean()


cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,255,0),2)

text = f'{cat_types[index_unit]}: {int(prob_avg*100)}'
cv2.putText(clone, text, (startx, starty), cv2.FONT_HERSHEY_SIMPLEX, .45, (0,255,0),2)
   
    
cv2.imshow('test', clone)
cv2.waitKey(0)

In [ ]:
assert False

In [ ]:
clone = img.copy()
  
startx, starty, endx, endy = new_loc[boxid]
cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,255,0),2)
text = f'Weapon: {i*100}'
cv2.putText(clone, text, (startx, y), cv2.FONT_HERSHEY_SIMPLEX, .45, (0,255,0),2)
cv2.imshow('test', clone)
cv2.waitKey(0)

In [ ]:
clone = img.copy()
roi = clone[starty:endy, startx:endx]
roi = cv2.resize(roi, dsize =var.dimension, interpolation = cv2.INTER_CUBIC)
# cv2.imshow('test', roi)
# cv2.waitKey(0)
explainer = lime_image.LimeImageExplainer()

explanation = explainer.explain_instance(roi, normal_model.predict, top_labels = 5, hide_color = 0, 
                                         num_samples = 1000)


temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only = False,
                                           num_features = 10, hide_rest = False)
plt.imshow(mark_boundaries(temp/2 + .5, mask))